# Introduction:
Mondial is a set of tables that cover basic information like location, border, economy, population and nature resources in a global basis. I select this database with the expectation to explore the state of  nature resource sustainability and its relation with economy.


# Question:
What are different countries’ states in terms of nature resource sustainability. Is there a relation between varied nature resource and GDP?

In [23]:
#importing the necessary libraries
import mysql.connector # this library is used to connect to our mysql database from our python environment
import pandas as pd # this library named pandas will be used to manipulate our data 

# defining the credentials
our_username = "root"
our_password = "" #this is a blank password
our_host = "localhost" #this is where the database is hosted, in this case it is hosted locally

# creating the connection to our mysql
conn = mysql.connector.connect(host=our_host, username=our_username, password=our_password)

# creating our database cursor that will allow us to execute SQL queries to our connected database
db_cursor = conn.cursor(buffered=True)

# checking if we are connected and ready to go
if conn.is_connected():
    print("Our connection to our database is successful")
else:
    print("Our connection to our database is unsuccessful")

Our connection to our database is successful


In [24]:
database_name = "mondial"

query = f"CREATE DATABASE IF NOT EXISTS {database_name}"

result = db_cursor.execute(query)

query_select_database = f"USE {database_name}"

result = db_cursor.execute(query_select_database)

Question1:
List countries and the water resources(lake/river/sea)they have by creating three new tables(lake1,river1,sea1);

In [18]:
#Combine geo information with resource table
sql_query = """
CREATE TABLE lake_g AS 
SELECT * FROM lake 
JOIN geo_lake ON lake.Name = geo_lake.Lake;

CREATE TABLE river_g AS 
SELECT river.Name, river.Length, geo_river.country 
FROM river 
JOIN geo_river ON river.Name = geo_river.River;

CREATE TABLE sea_g AS 
SELECT * FROM sea 
JOIN geo_sea ON sea.Name = geo_sea.Sea;
"""
pd.read_sql(sql_query, conn)

TypeError: 'NoneType' object is not iterable

In [25]:
# Join tables into one;
sql_query = """
SELECT DISTINCT lake_g.Name AS lake_name, 
       lake_g.Depth AS lake_depth, 
       sea_g.Name AS sea_name, 
       sea_g.Depth AS sea_depth, 
       river_g.Name AS river_name, 
       river_g.Length AS river_length 
FROM lake_g JOIN sea_g 
ON lake_g.Country = sea_g.Country 
JOIN river_g 
ON river_g.Country = lake_g.Country 
ORDER BY lake_g.Country;
"""

pd.read_sql(sql_query, conn)

,lake_name,lake_depth,sea_name,sea_depth,river_name,river_length
0,Lake Ohrid,289.0,Mediterranean Sea,5121.0,Black Drin,133.0
1,Lake Skutari,44.0,Mediterranean Sea,5121.0,Black Drin,133.0
2,Lake Prespa,54.0,Mediterranean Sea,5121.0,Drin,152.0
3,Lake Ohrid,289.0,Mediterranean Sea,5121.0,White Drin,175.0
4,Lake Skutari,44.0,Mediterranean Sea,5121.0,White Drin,175.0
...,...,...,...,...,...,...
5016,Rutanzige/Eduardsee,112.0,Atlantic Ocean,9219.0,Ruki,130.0
5017,Lake Mai Ndombe,10.0,Atlantic Ocean,9219.0,Tshuapa,1300.0
5018,Lake Mai Ndombe,10.0,Atlantic Ocean,9219.0,Ruzizi,104.0
5019,Lake Kivu,450.0,Atlantic Ocean,9219.0,Ruzizi,104.0


Question2:
Find countries that are located on more than one continent.

In [26]:
sql_query = """
SELECT en1.Country, en1.Continent, en1.Percentage, en2.Continent, en2.Percentage
FROM encompasses AS en1 JOIN encompasses AS en2
ON en1.Country = en2.Country
WHERE en1.Continent <> en2.Continent
GROUP BY Country
"""

pd.read_sql(sql_query, conn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Country,Continent,Percentage,Continent,Percentage
0,ET,Africa,90.0,Asia,10.0
1,R,Asia,80.0,Europe,20.0
2,RI,Asia,80.0,Australia/Oceania,20.0
3,TR,Asia,95.0,Europe,5.0


Question3:
Find countries with diversified non-water resources(mountain/island/desert) 

In [27]:
#Join tables and count no-water resource;
sql_query = """
SELECT country.Name, nwr.non_water_resource 
FROM country 
JOIN 
(SELECT DISTINCT Country, COUNT(*)AS non_water_resource 
FROM geo_desert 
GROUP BY Country 
UNION SELECT DISTINCT Country,COUNT(*)AS non_water_resource 
FROM geo_mountain 
GROUP BY Country 
UNION SELECT DISTINCT Country,COUNT(*)AS non_water_resource 
FROM geo_island 
GROUP BY Country) nwr 
ON country.Code = nwr.country 
GROUP BY country.Name 
HAVING non_water_resource > 10
ORDER BY nwr.non_water_resource 
DESC;
"""

pd.read_sql(sql_query, conn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Name,non_water_resource
0,United Kingdom,96
1,Japan,47
2,United States,15
3,Germany,15
4,Indonesia,15
5,Cuba,13
6,Canada,11
7,Portugal,11
8,Spain,11
9,Italy,11


Question4:
Find top 10 countries with the higheset lake proportion.

In [28]:
# Combine lake and its geo information to a new table: lakelist
sql_query = """
CREATE TABLE lakelist AS
(SELECT DISTINCT lake.Name, geo_lake.Country, lake.Area, lake.Depth, lake.Altitude, lake.Type 
 FROM lake 
 JOIN geo_lake 
 ON lake.Name = geo_lake.Lake 
 ORDER BY geo_lake.Country);
"""

pd.read_sql(sql_query, conn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


TypeError: 'NoneType' object is not iterable

In [29]:
#4 Add country infomation to lakelist;
sql_query = """
CREATE TABLE country_lake AS 
(SELECT DISTINCT country.Name, country.Area AS country_area, country.Population AS country_population, 
                 lakelist.Name AS lake_name, lakelist.Area AS lake_area, lakelist.Depth AS lake_depth, 
                 lakelist.Altitude AS lake_altitude, lakelist.Type AS lake_type 
FROM lakelist JOIN country 
ON lakelist.Country = country.Code)
"""

pd.read_sql(sql_query, conn)


TypeError: 'NoneType' object is not iterable

In [30]:
#4 Count lake_proportion
sql_query = """
SELECT country_lake.Name, country_lake.country_area, country_lake.lake_area, 
ROUND( country_lake.lake_area / country_lake.country_area, 3) AS lake_proportion 
FROM country_lake
GROUP BY country_lake.Name 
ORDER BY lake_proportion 
DESC 
LIMIT 10
"""

pd.read_sql(sql_query, conn)

,Name,country_area,lake_area,lake_proportion
0,West Bank,5860.0,41650.0,7.108
1,Azerbaijan,86600.0,386400.0,4.462
2,Burundi,27830.0,32893.0,1.182
3,Turkmenistan,488100.0,386400.0,0.792
4,Jordan,89213.0,41650.0,0.467
5,Costa Rica,51100.0,8157.0,0.160
6,Kenya,582650.0,68870.0,0.118
7,Rwanda,26340.0,2650.0,0.101
8,Estonia,45100.0,3555.0,0.079
9,Nicaragua,129494.0,8157.0,0.063


Question5:
High danger nations:highly populated countries with little water area;

In [31]:
# Create a new table:country_lake_per_person to count lake per person;
sql_query = """
CREATE TABLE country_lake_per_person AS 
(SELECT country_lake.Name, country_lake.country_population, 
        ROUND(country_lake.lake_area/country_lake.country_population, 10) AS lake_per_person 
FROM country_lake 
GROUP BY country_lake.Name 
ORDER BY lake_per_person ASC 
LIMIT 50)
"""

pd.read_sql(sql_query, conn)


/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


TypeError: 'NoneType' object is not iterable

In [32]:
#5 select
sql_query = """
SELECT * 
FROM country_lake_per_person
"""

pd.read_sql(sql_query, conn)

,Name,country_population,lake_per_person
0,Cameroon,14261557,NaN
1,Chile,14333258,3.000000e-10
2,Bolivia,7165257,7.000000e-10
3,United Kingdom,58489975,9.574000e-07
4,Italy,57460274,1.984000e-06
5,United States,266476278,2.401700e-06
6,China,1210004956,3.787600e-06
7,Indonesia,206611600,5.338500e-06
8,Ethiopia,57171662,5.597200e-06
9,Germany,83536115,6.446300e-06


Question6:
Potential watershortage: nations have desert area larger than the average;

In [33]:
# Link country and desert
sql_query = """
CREATE TABLE country_desert AS 
(SELECT DISTINCT country.Name, country.Area, country.Population, dg.desert_name, dg.desert_area 
FROM country  
RIGHT JOIN 
(SELECT DISTINCT desert.Name AS desert_name, desert.Area AS desert_area, geo_desert.Country AS country_code 
FROM desert JOIN geo_desert 
ON geo_desert.Desert = desert.Name) dg
ON country.Code= dg.country_code
GROUP BY country.Name)
"""

pd.read_sql(sql_query, conn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


TypeError: 'NoneType' object is not iterable

In [34]:
#6 Select countries with desert area larger than the average
sql_query = """
SELECT Name, desert_name, desert_area
FROM country_desert
WHERE desert_area > (SELECT AVG(desert_area) FROM country_desert)
ORDER BY desert_area DESC
"""

pd.read_sql(sql_query, conn)


,Name,desert_name,desert_area
0,Angola,Kalahari,1200000.0
1,South Africa,Kalahari,1200000.0
2,Namibia,Kalahari,1200000.0
3,Zambia,Kalahari,1200000.0
4,Botswana,Kalahari,1200000.0
5,Egypt,Libyan Desert,1100000.0
6,Sudan,Libyan Desert,1100000.0
7,China,Gobi,1000000.0
8,Mongolia,Gobi,1000000.0
9,Oman,Rub Al Chali,780000.0


Question7:
List countries and count Lake per person 

In [35]:
sql_query = """
SELECT c.Name, c.Code, l.Name, l.Area/ c.Population AS lake_perperson
FROM country AS c LEFT JOIN lake_g AS l 
ON c.Code = l.Country
GROUP BY c.Code
ORDER BY l.Area/ c.Population DESC
"""

pd.read_sql(sql_query, conn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Name,Code,Name,lake_perperson
0,Turkmenistan,TM,Caspian Sea,0.093125
1,Azerbaijan,AZ,Caspian Sea,0.050332
2,West Bank,WEST,Dead Sea,0.029172
3,Jordan,JOR,Dead Sea,0.009888
4,Burundi,BI,Lake Tanganjika,0.005535
...,...,...,...,...
233,Guam,GUAM,None,NaN
234,Marshall Islands,MH,None,NaN
235,Haiti,RH,None,NaN
236,Trinidad and Tobago,TT,None,NaN


Question8:
Find top 20 countries with their GDP and proportions of three industries; 

In [36]:
sql_query = """
SELECT c.Name, c.Code, e.GDP, e.Agriculture, e.Service, e.Industry
FROM country AS c JOIN economy AS e
ON c.Code = e.Country
ORDER BY e.GDP DESC
LIMIT 20
"""

pd.read_sql(sql_query, conn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Name,Code,GDP,Agriculture,Service,Industry
0,United States,USA,7247700.0,2.0,23.0,75.0
1,China,TJ,3500000.0,19.0,48.0,33.0
2,Japan,J,2679200.0,2.1,40.2,57.7
3,Germany,D,1452200.0,1.0,34.2,64.8
4,India,IND,1408700.0,NaN,NaN,NaN
5,France,F,1173000.0,2.4,26.5,71.1
6,United Kingdom,GB,1138400.0,1.7,27.7,70.6
7,Italy,I,1088600.0,2.9,31.6,65.5
8,Brazil,BR,976800.0,16.0,25.0,59.0
9,Russia,R,796000.0,6.0,41.0,53.0


Question9:
Find countries that gdp per capita >0.1

In [37]:
sql_query = """
SELECT c.Name, c.Code, c.Area/ c.Population AS per_capita_area, e.GDP
FROM country AS c JOIN economy AS e
ON c.Code = e.Country
GROUP BY c.Code 
HAVING per_capita_area > 0.1
ORDER BY per_capita_area DESC
"""

pd.read_sql(sql_query, conn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Name,Code,per_capita_area,GDP
0,Greenland,GROX,37.379517,892.00
1,Svalbard,SVAX,29.323724,NaN
2,Falkland Islands,FALK,3.876752,105.10
3,Western Sahara,WSA,1.194802,NaN
4,Pitcairn Islands,PITC,0.979167,NaN
5,Mongolia,MNG,0.626848,4900.00
6,French Guiana,FGU,0.601904,800.00
7,Namibia,NAM,0.492128,5800.00
8,Mauritania,RIM,0.441215,2800.00
9,Australia,AUS,0.420947,405400.00


Question10:
Find if artificial lakes of a country affect the GDP;

In [38]:
sql_query = """
SELECT DISTINCT gl.Lake, gl.Country, la.Type, la.Area, e.GDP
FROM geo_lake AS gl JOIN lake AS la
ON gl.Lake = la.Name 
JOIN economy AS e 
ON gl.Country = e.Country
WHERE la.Type = 'artificial'
ORDER BY e.GDP DESC;
"""

pd.read_sql(sql_query, conn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Lake,Country,Type,Area,GDP
0,Fort Peck Lake,USA,artificial,981.00,7247700.0
1,Lake Mead,USA,artificial,640.00,7247700.0
2,Lake Sakakawea,USA,artificial,1578.00,7247700.0
3,Lake Powell,USA,artificial,645.00,7247700.0
4,Franklin. D. Roosevelt Lake,USA,artificial,337.00,7247700.0
5,Lake Oahe,USA,artificial,1453.00,7247700.0
6,Lago de Sobradinho,BR,artificial,4214.00,976800.0
7,Lake Burley Griffin,AUS,artificial,6.64,405400.0
8,Lake Nasser,ET,artificial,5250.00,171000.0
9,Lake Kainji,WAN,artificial,1243.00,135900.0


# Research Question: 
What are different countries’ states in terms of nature resource sustainability? Is there a relation between varied nature resource and GDP?

# Findings:
I list countries with abundant water and non-water resource and infer countries which may face potential nature resource shortage based on the low value of water resource per person and larger area of desert. I find out that countries with sufficient non-water resource may also face with potential water shortage(i.e. UK and USA)
No obvious relationship between the nature resource and GDP.
However, the larger the proportion of industry, the higher the level of economic development

# Limitation:
1.The water resource has different measuring scales (depth/length/area). We can’t summerize by the aggregation 
of length and depth.

2.Data is outdated (mainly in 1998 and 2015).

3.Only one year GDP resource, no horizontally analyzed.

4.Other factors like import/export/multinational exchange may affect sustainability and GDP but are not included in the database.